# Prediction of used car prices 

Loading required libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

from sklearn.metrics import r2_score


# Import Data

In [2]:
df_tr=pd.read_csv('data_train.csv')
df_te=pd.read_csv('data_test.csv')
df_tr.shape, df_te.shape

((50000, 17), (27000, 17))

In [3]:
df_tr.head()

,manufacturer_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_type,engine_capacity,body_type,has_warranty,ownership,type_of_drive,is_exchangeable,number_of_photos,number_of_maintenance,duration_listed,price_usd
0,Volkswagen,automatic,black,130000,2016,diesel,diesel,1.6,universal,False,owned,front,True,17,38,67,13150.0
1,Renault,manual,brown,149000,2012,gasoline,gasoline,1.6,sedan,False,owned,front,False,9,3,100,7500.0
2,Kia,automatic,brown,110000,2014,gasoline,gasoline,1.6,hatchback,False,owned,front,False,5,10,91,12200.0
3,Opel,automatic,other,255100,2007,gasoline,gasoline,1.8,hatchback,False,owned,front,False,10,4,91,4950.0
4,Mazda,manual,blue,650000,1999,gasoline,gasoline,2.0,sedan,False,owned,front,True,5,7,62,3000.0


Checking null values

In [4]:
df_tr.isnull().sum()

manufacturer_name         0
transmission              0
color                     0
odometer_value            0
year_produced             0
engine_fuel               0
engine_type               0
engine_capacity          15
body_type                 0
has_warranty              0
ownership                 0
type_of_drive             0
is_exchangeable           0
number_of_photos          0
number_of_maintenance     0
duration_listed           0
price_usd                 0
dtype: int64

In [5]:
df_te.isnull().sum()

manufacturer_name        0
transmission             0
color                    0
odometer_value           0
year_produced            0
engine_fuel              0
engine_type              0
engine_capacity          5
body_type                0
has_warranty             0
ownership                0
type_of_drive            0
is_exchangeable          0
number_of_photos         0
number_of_maintenance    0
duration_listed          0
price_usd                0
dtype: int64

Dropping na values

In [6]:
dataTrain=df_tr.dropna()
dataTest=df_te.dropna()

In [7]:
dataTrain.isnull().sum()

manufacturer_name        0
transmission             0
color                    0
odometer_value           0
year_produced            0
engine_fuel              0
engine_type              0
engine_capacity          0
body_type                0
has_warranty             0
ownership                0
type_of_drive            0
is_exchangeable          0
number_of_photos         0
number_of_maintenance    0
duration_listed          0
price_usd                0
dtype: int64

In [8]:
dataTrain.isnull().sum()

manufacturer_name        0
transmission             0
color                    0
odometer_value           0
year_produced            0
engine_fuel              0
engine_type              0
engine_capacity          0
body_type                0
has_warranty             0
ownership                0
type_of_drive            0
is_exchangeable          0
number_of_photos         0
number_of_maintenance    0
duration_listed          0
price_usd                0
dtype: int64

In [ ]:
dataTest.isnull().sum()

manufacturer_name        0
transmission             0
color                    0
odometer_value           0
year_produced            0
engine_fuel              0
engine_type              0
engine_capacity          0
body_type                0
has_warranty             0
ownership                0
type_of_drive            0
is_exchangeable          0
number_of_photos         0
number_of_maintenance    0
duration_listed          0
price_usd                0
dtype: int64

In [10]:
dataTrain.shape, dataTest.shape

((49985, 17), (26995, 17))

In [11]:
dataTrain.dtypes

manufacturer_name         object
transmission              object
color                     object
odometer_value             int64
year_produced              int64
engine_fuel               object
engine_type               object
engine_capacity          float64
body_type                 object
has_warranty                bool
ownership                 object
type_of_drive             object
is_exchangeable             bool
number_of_photos           int64
number_of_maintenance      int64
duration_listed            int64
price_usd                float64
dtype: object

In [12]:
dataTrain.describe()

,odometer_value,year_produced,engine_capacity,number_of_photos,number_of_maintenance,duration_listed,price_usd
count,49985.000000,49985.000000,49985.000000,49985.000000,49985.000000,49985.000000,49985.000000
mean,249275.754426,2002.942143,2.056894,9.660958,16.257417,80.749385,6651.131653
std,136342.510114,8.065809,0.675106,6.144711,41.542444,113.620345,6462.573155
min,0.000000,1942.000000,0.200000,1.000000,1.000000,0.000000,1.000000
25%,158000.000000,1998.000000,1.600000,5.000000,2.000000,23.000000,2100.000000
50%,250000.000000,2003.000000,2.000000,8.000000,5.000000,59.000000,4800.000000
75%,325000.000000,2009.000000,2.300000,12.000000,16.000000,91.000000,8963.290000
max,1000000.000000,2019.000000,8.000000,86.000000,1273.000000,2232.000000,50000.000000


In [13]:
dataTest.describe()

,odometer_value,year_produced,engine_capacity,number_of_photos,number_of_maintenance,duration_listed,price_usd
count,26995.000000,26995.000000,26995.000000,26995.000000,26995.000000,26995.000000,26995.000000
mean,248253.283497,2002.935655,2.052027,9.623597,16.387776,80.270828,6610.308694
std,135573.549451,8.063814,0.663980,5.997012,46.373189,111.433261,6357.865198
min,0.000000,1942.000000,0.200000,1.000000,1.000000,0.000000,1.000000
25%,157000.000000,1998.000000,1.600000,5.000000,2.000000,23.000000,2150.000000
50%,249000.000000,2003.000000,2.000000,8.000000,5.000000,60.000000,4799.000000
75%,324000.000000,2009.000000,2.300000,12.000000,16.000000,91.000000,8950.000000
max,1000000.000000,2019.000000,8.000000,71.000000,1861.000000,2175.000000,50000.000000


Data preprosessing 

In [14]:
X_train = pd.get_dummies(dataTrain, drop_first=True)
X_test = pd.get_dummies(dataTest, drop_first=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

In [15]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Converting boolen to int

In [16]:
# Identify boolean columns
bool_columns = X_train.select_dtypes(include='bool').columns

# Convert those columns to integers
X_train[bool_columns] = X_train[bool_columns].astype(int)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49985 entries, 0 to 49999
Data columns (total 95 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   odometer_value                   49985 non-null  int64  
 1   year_produced                    49985 non-null  int64  
 2   engine_capacity                  49985 non-null  float64
 3   has_warranty                     49985 non-null  int32  
 4   is_exchangeable                  49985 non-null  int32  
 5   number_of_photos                 49985 non-null  int64  
 6   number_of_maintenance            49985 non-null  int64  
 7   duration_listed                  49985 non-null  int64  
 8   price_usd                        49985 non-null  float64
 9   manufacturer_name_Alfa Romeo     49985 non-null  int32  
 10  manufacturer_name_Audi           49985 non-null  int32  
 11  manufacturer_name_BMW            49985 non-null  int32  
 12  manufacturer_name_Buick

In [17]:
# Identify boolean columns
bool_columns = X_test.select_dtypes(include='bool').columns

# Convert those columns to integers
X_test[bool_columns] = X_test[bool_columns].astype(int)
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26995 entries, 0 to 26999
Data columns (total 95 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   odometer_value                   26995 non-null  int64  
 1   year_produced                    26995 non-null  int64  
 2   engine_capacity                  26995 non-null  float64
 3   has_warranty                     26995 non-null  int32  
 4   is_exchangeable                  26995 non-null  int32  
 5   number_of_photos                 26995 non-null  int64  
 6   number_of_maintenance            26995 non-null  int64  
 7   duration_listed                  26995 non-null  int64  
 8   price_usd                        26995 non-null  float64
 9   manufacturer_name_Alfa Romeo     26995 non-null  int32  
 10  manufacturer_name_Audi           26995 non-null  int32  
 11  manufacturer_name_BMW            26995 non-null  int32  
 12  manufacturer_name_Buick

In [18]:
X_train1=X_train.drop(columns=['price_usd'])
y_train=X_train['price_usd']
#test set
X_test1=X_test.drop(columns=['price_usd'])
y_test=X_test['price_usd']

In [19]:
X_train1.shape, y_train.shape, X_test1.shape, y_test.shape

((49985, 94), (49985,), (26995, 94), (26995,))

In [20]:
from sklearn.model_selection import train_test_split

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train1, y_train, test_size=0.05, random_state=42)


In [21]:
X_train_split.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47485 entries, 25051 to 15799
Data columns (total 94 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   odometer_value                   47485 non-null  int64  
 1   year_produced                    47485 non-null  int64  
 2   engine_capacity                  47485 non-null  float64
 3   has_warranty                     47485 non-null  int32  
 4   is_exchangeable                  47485 non-null  int32  
 5   number_of_photos                 47485 non-null  int64  
 6   number_of_maintenance            47485 non-null  int64  
 7   duration_listed                  47485 non-null  int64  
 8   manufacturer_name_Alfa Romeo     47485 non-null  int32  
 9   manufacturer_name_Audi           47485 non-null  int32  
 10  manufacturer_name_BMW            47485 non-null  int32  
 11  manufacturer_name_Buick          47485 non-null  int32  
 12  manufacturer_name_C

In [22]:
X_train_split.shape,y_train_split.shape,X_val_split.shape,y_val_split.shape,X_test1.shape,y_test.shape

((47485, 94), (47485,), (2500, 94), (2500,), (26995, 94), (26995,))

Model building

In [23]:
tf.random.set_seed(42)

# Build the model
model = tf.keras.Sequential()

# Input layer (automatically determined by input shape)
model.add(tf.keras.layers.InputLayer(input_shape=(X_train1.shape[1],)))

# Hidden layers
model.add(tf.keras.layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))  # First hidden layer
model.add(tf.keras.layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))  # Second hidden layer
model.add(tf.keras.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
model.add(tf.keras.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))

# Output layer
model.add(tf.keras.layers.Dense(1))  # For regression, no activation function is used for the output layer

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00075)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])


In [24]:
# Train the model
history = model.fit(X_train_split, y_train_split, 
                    epochs=80, 
                    batch_size=128, 
                    validation_data=(X_val_split, y_val_split))


Epoch 1/80
371/371 [==============================] - 4s 5ms/step - loss: 49129996.0000 - mae: 4706.0825 - val_loss: 29584982.0000 - val_mae: 3749.8127
Epoch 2/80
371/371 [==============================] - 2s 5ms/step - loss: 32670852.0000 - mae: 3913.3135 - val_loss: 29244338.0000 - val_mae: 3735.0564
Epoch 3/80
371/371 [==============================] - 2s 5ms/step - loss: 32183532.0000 - mae: 3864.2214 - val_loss: 32462828.0000 - val_mae: 3828.9880
Epoch 4/80
371/371 [==============================] - 2s 5ms/step - loss: 32018106.0000 - mae: 3860.6040 - val_loss: 29820728.0000 - val_mae: 3699.9053
Epoch 5/80
371/371 [==============================] - 2s 5ms/step - loss: 31746682.0000 - mae: 3832.2327 - val_loss: 29023898.0000 - val_mae: 3833.2312
Epoch 6/80
371/371 [==============================] - 2s 5ms/step - loss: 31474828.0000 - mae: 3814.5544 - val_loss: 30362946.0000 - val_mae: 4162.0610
Epoch 7/80
371/371 [==============================] - 2s 5ms/step - loss: 31167490.0000 

Compiling the model

In [25]:


# Get predictions on the test set
y_test_pred = model.predict(X_test1).flatten()  # Flatten to ensure 1D array

# Convert y_test to NumPy array (if it's a Pandas Series)
y_test_np = np.array(y_test)

# Compute R² score on test set
r2_test = r2_score(y_test_np, y_test_pred)
print("R² Score on Test Data:", r2_test)


844/844 [==============================] - 2s 2ms/step
R² Score on Test Data: 0.64406574819529


In [26]:


# Get predictions on validation data (already a NumPy array)
y_val_pred = model.predict(X_val_split).flatten()  # Ensure it's a 1D array

# Convert y_valid to a NumPy array (if it's a Pandas Series)
y_valid_np = np.array(y_val_split)

# Compute R² score
r2 = r2_score(y_valid_np, y_val_pred)
print("R² Score on Validation Data:", r2)


79/79 [==============================] - 0s 2ms/step
R² Score on Validation Data: 0.6043826994273249
